# Trades Notebook

In [7]:
import psycopg2
import pandas as pd

# create a connection to the Postgres database
connection = psycopg2.connect("dbname=postgres host=sp-psql user=postgres password=postgres")
# create a cursor
cursor = connection.cursor()

# execute a query
cursor.execute("SELECT * FROM public.trades")

# retrieve query results
records = cursor.fetchall()
cursor.close()

column_names = ['account_id','symbol','side','qty','timestamp']
trades_df = pd.DataFrame(records, columns=column_names)
print(trades_df)

       account_id symbol  side  qty           timestamp
0           77658   GOOG  sell  493 2023-09-04 09:30:00
1           19191   NVDA  sell  216 2023-09-04 09:30:03
2           90209   NVDA  sell  108 2023-09-04 09:30:07
3           59797     FB   buy  612 2023-09-04 09:30:07
4           20249   NFLX   buy    7 2023-09-04 09:30:12
...           ...    ...   ...  ...                 ...
37647       38363   NFLX  sell  732 2023-09-08 15:59:35
37648       19170   MSFT   buy  726 2023-09-08 15:59:37
37649       18089   GOOG  sell  363 2023-09-08 15:59:51
37650       88273   NFLX   buy   11 2023-09-08 15:59:56
37651       28035   GOOG  sell   26 2023-09-08 15:59:57

[37652 rows x 5 columns]
